Original code from https://github.com/eriklindernoren/Keras-GAN/blob/master/dcgan/dcgan.py

In [1]:
images_dir = "dcgan_images"

In [3]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class DCGAN():
    def __init__(self):
        self.img_rows = 28 
        self.img_cols = 28
        self.channels = 1

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy', 
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)
        
        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_shape=noise_shape))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8)) 
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(1, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)
        
        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        half_batch = int(batch_size / 2)

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            # Sample noise and generate a half batch of new images
            noise = np.random.normal(0, 1, (half_batch, 100))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, np.ones((batch_size, 1)))

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, 100))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        #fig.suptitle("DCGAN: Generated digits", fontsize=12)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(os.path.join(images_dir, 'mnist_{}.png'.format(epoch)))
        plt.close()

In [ ]:
# set up directories
import os

if (not os.path.isdir(images_dir)):
    os.mkdir(images_dir)

In [4]:
dcgan = DCGAN()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

In [5]:
dcgan.train(epochs=4000, batch_size=32, save_interval=50)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/keras/engine/training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.978809, acc.: 34.38%] [G loss: 1.009362]
1 [D loss: 0.848249, acc.: 43.75%] [G loss: 0.769924]
2 [D loss: 0.471850, acc.: 84.38%] [G loss: 0.797407]
3 [D loss: 0.291681, acc.: 96.88%] [G loss: 0.921089]
4 [D loss: 0.182269, acc.: 100.00%] [G loss: 0.883840]
5 [D loss: 0.272387, acc.: 90.62%] [G loss: 1.019941]
6 [D loss: 0.528635, acc.: 65.62%] [G loss: 0.992732]
7 [D loss: 0.408201, acc.: 78.12%] [G loss: 1.290603]
8 [D loss: 0.583973, acc.: 65.62%] [G loss: 1.602742]
9 [D loss: 0.608605, acc.: 59.38%] [G loss: 1.593753]
10 [D loss: 0.542754, acc.: 71.88%] [G loss: 1.386537]
11 [D loss: 0.363704, acc.: 81.25%] [G loss: 1.113673]
12 [D loss: 0.418573, acc.: 87.50%] [G loss: 0.987514]
13 [D loss: 0.245698, acc.: 93.75%] [G loss: 0.571558]
14 [D loss: 0.446392, acc.: 78.12%] [G loss: 0.539497]
15 [D loss: 0.314833, acc.: 87.50%] [G loss: 0.497540]
16 [D loss: 0.256997, acc.: 96.88%] [G loss: 0.338486]
17 [D loss: 0.267278, acc.: 93.75%] [G loss: 0.198734]
18 [D loss: 0.30116

149 [D loss: 0.845317, acc.: 46.88%] [G loss: 1.560692]
150 [D loss: 1.006998, acc.: 37.50%] [G loss: 1.207491]
151 [D loss: 0.977671, acc.: 40.62%] [G loss: 1.429054]
152 [D loss: 0.762078, acc.: 65.62%] [G loss: 1.335081]
153 [D loss: 1.115793, acc.: 40.62%] [G loss: 1.123638]
154 [D loss: 0.891755, acc.: 50.00%] [G loss: 1.295190]
155 [D loss: 0.934308, acc.: 46.88%] [G loss: 1.261819]
156 [D loss: 1.307135, acc.: 21.88%] [G loss: 1.026778]
157 [D loss: 0.819353, acc.: 46.88%] [G loss: 1.219687]
158 [D loss: 0.855971, acc.: 40.62%] [G loss: 1.312308]
159 [D loss: 0.978156, acc.: 43.75%] [G loss: 0.996085]
160 [D loss: 0.803571, acc.: 43.75%] [G loss: 1.208792]
161 [D loss: 0.923196, acc.: 37.50%] [G loss: 1.204777]
162 [D loss: 0.970870, acc.: 37.50%] [G loss: 1.106678]
163 [D loss: 0.989489, acc.: 53.12%] [G loss: 0.865835]
164 [D loss: 0.908859, acc.: 53.12%] [G loss: 1.168872]
165 [D loss: 0.783120, acc.: 65.62%] [G loss: 1.075397]
166 [D loss: 0.658778, acc.: 68.75%] [G loss: 1.

296 [D loss: 0.829231, acc.: 50.00%] [G loss: 1.031808]
297 [D loss: 0.825654, acc.: 50.00%] [G loss: 1.077005]
298 [D loss: 0.782262, acc.: 56.25%] [G loss: 1.088068]
299 [D loss: 0.916525, acc.: 40.62%] [G loss: 1.051188]
300 [D loss: 0.860624, acc.: 40.62%] [G loss: 1.035603]
301 [D loss: 0.571410, acc.: 75.00%] [G loss: 1.185880]
302 [D loss: 0.657239, acc.: 65.62%] [G loss: 1.077002]
303 [D loss: 0.723909, acc.: 56.25%] [G loss: 1.048730]
304 [D loss: 0.837889, acc.: 46.88%] [G loss: 1.046008]
305 [D loss: 0.819729, acc.: 43.75%] [G loss: 0.947805]
306 [D loss: 0.823740, acc.: 40.62%] [G loss: 1.231177]
307 [D loss: 0.726778, acc.: 53.12%] [G loss: 0.870745]
308 [D loss: 0.907055, acc.: 50.00%] [G loss: 1.079357]
309 [D loss: 0.876176, acc.: 31.25%] [G loss: 1.125926]
310 [D loss: 0.679735, acc.: 62.50%] [G loss: 1.157826]
311 [D loss: 0.712602, acc.: 56.25%] [G loss: 1.153126]
312 [D loss: 0.757853, acc.: 43.75%] [G loss: 0.920831]
313 [D loss: 0.791472, acc.: 50.00%] [G loss: 0.

443 [D loss: 0.661937, acc.: 50.00%] [G loss: 1.006398]
444 [D loss: 0.685962, acc.: 65.62%] [G loss: 0.924920]
445 [D loss: 0.533969, acc.: 81.25%] [G loss: 1.062316]
446 [D loss: 0.876464, acc.: 34.38%] [G loss: 1.054227]
447 [D loss: 0.764924, acc.: 43.75%] [G loss: 1.052603]
448 [D loss: 0.756044, acc.: 46.88%] [G loss: 0.807976]
449 [D loss: 0.680266, acc.: 71.88%] [G loss: 0.892229]
450 [D loss: 0.675105, acc.: 65.62%] [G loss: 0.904104]
451 [D loss: 0.785706, acc.: 46.88%] [G loss: 0.801820]
452 [D loss: 0.709674, acc.: 56.25%] [G loss: 0.985483]
453 [D loss: 0.716787, acc.: 50.00%] [G loss: 1.025079]
454 [D loss: 0.855031, acc.: 37.50%] [G loss: 1.025565]
455 [D loss: 0.665836, acc.: 56.25%] [G loss: 0.910964]
456 [D loss: 0.726275, acc.: 53.12%] [G loss: 0.961691]
457 [D loss: 0.899105, acc.: 43.75%] [G loss: 0.869168]
458 [D loss: 0.613314, acc.: 65.62%] [G loss: 1.052143]
459 [D loss: 0.692486, acc.: 62.50%] [G loss: 1.304182]
460 [D loss: 0.803660, acc.: 53.12%] [G loss: 0.

...

3334 [D loss: 0.625858, acc.: 62.50%] [G loss: 0.784853]
3335 [D loss: 0.648923, acc.: 56.25%] [G loss: 1.043878]
3336 [D loss: 0.626890, acc.: 62.50%] [G loss: 0.840457]
3337 [D loss: 0.595872, acc.: 65.62%] [G loss: 0.833200]
3338 [D loss: 0.715857, acc.: 53.12%] [G loss: 0.923538]
3339 [D loss: 0.701450, acc.: 56.25%] [G loss: 0.854229]
3340 [D loss: 0.569205, acc.: 75.00%] [G loss: 0.968072]
3341 [D loss: 0.821257, acc.: 46.88%] [G loss: 0.771982]
3342 [D loss: 0.789489, acc.: 43.75%] [G loss: 0.747578]
3343 [D loss: 0.655137, acc.: 56.25%] [G loss: 0.773741]
3344 [D loss: 0.816749, acc.: 43.75%] [G loss: 0.795865]
3345 [D loss: 0.622243, acc.: 65.62%] [G loss: 1.071618]
3346 [D loss: 0.644905, acc.: 59.38%] [G loss: 1.005727]
3347 [D loss: 0.648051, acc.: 65.62%] [G loss: 0.903635]
3348 [D loss: 0.596021, acc.: 65.62%] [G loss: 0.947560]
3349 [D loss: 0.653255, acc.: 62.50%] [G loss: 0.969816]
3350 [D loss: 0.646585, acc.: 56.25%] [G loss: 0.942195]
3351 [D loss: 0.682477, acc.: 5

3478 [D loss: 0.554437, acc.: 75.00%] [G loss: 0.811959]
3479 [D loss: 0.719858, acc.: 53.12%] [G loss: 1.019763]
3480 [D loss: 0.643036, acc.: 59.38%] [G loss: 0.982000]
3481 [D loss: 0.660810, acc.: 62.50%] [G loss: 1.030482]
3482 [D loss: 0.700990, acc.: 53.12%] [G loss: 1.036109]
3483 [D loss: 0.697782, acc.: 50.00%] [G loss: 0.945360]
3484 [D loss: 0.684924, acc.: 53.12%] [G loss: 0.811394]
3485 [D loss: 0.664976, acc.: 50.00%] [G loss: 0.756405]
3486 [D loss: 0.648034, acc.: 62.50%] [G loss: 0.835452]
3487 [D loss: 0.530213, acc.: 71.88%] [G loss: 0.922693]
3488 [D loss: 0.770804, acc.: 46.88%] [G loss: 0.922423]
3489 [D loss: 0.567994, acc.: 68.75%] [G loss: 1.035015]
3490 [D loss: 0.584915, acc.: 71.88%] [G loss: 0.942102]
3491 [D loss: 0.650470, acc.: 68.75%] [G loss: 0.957830]
3492 [D loss: 0.744381, acc.: 59.38%] [G loss: 0.855663]
3493 [D loss: 0.730699, acc.: 53.12%] [G loss: 0.926932]
3494 [D loss: 0.749983, acc.: 50.00%] [G loss: 0.865228]
3495 [D loss: 0.738669, acc.: 5

3622 [D loss: 0.680594, acc.: 59.38%] [G loss: 0.992052]
3623 [D loss: 0.659833, acc.: 56.25%] [G loss: 0.912598]
3624 [D loss: 0.777009, acc.: 46.88%] [G loss: 0.936828]
3625 [D loss: 0.763918, acc.: 53.12%] [G loss: 0.766180]
3626 [D loss: 0.674688, acc.: 62.50%] [G loss: 0.829063]
3627 [D loss: 0.771541, acc.: 53.12%] [G loss: 0.838726]
3628 [D loss: 0.730500, acc.: 59.38%] [G loss: 1.101631]
3629 [D loss: 0.732699, acc.: 46.88%] [G loss: 0.886263]
3630 [D loss: 0.803394, acc.: 50.00%] [G loss: 0.836710]
3631 [D loss: 0.700217, acc.: 53.12%] [G loss: 1.056299]
3632 [D loss: 0.642717, acc.: 59.38%] [G loss: 0.893778]
3633 [D loss: 0.669495, acc.: 56.25%] [G loss: 0.923034]
3634 [D loss: 0.668547, acc.: 59.38%] [G loss: 0.980544]
3635 [D loss: 0.690920, acc.: 59.38%] [G loss: 0.897595]
3636 [D loss: 0.729989, acc.: 59.38%] [G loss: 1.001389]
3637 [D loss: 0.627792, acc.: 65.62%] [G loss: 0.830874]
3638 [D loss: 0.682647, acc.: 62.50%] [G loss: 0.884906]
3639 [D loss: 0.782805, acc.: 4

3766 [D loss: 0.650163, acc.: 65.62%] [G loss: 0.920140]
3767 [D loss: 0.747138, acc.: 59.38%] [G loss: 1.002551]
3768 [D loss: 0.655286, acc.: 56.25%] [G loss: 1.005338]
3769 [D loss: 0.742414, acc.: 50.00%] [G loss: 0.881153]
3770 [D loss: 0.780491, acc.: 43.75%] [G loss: 0.929822]
3771 [D loss: 0.696164, acc.: 50.00%] [G loss: 0.856184]
3772 [D loss: 0.662148, acc.: 62.50%] [G loss: 1.095547]
3773 [D loss: 0.817825, acc.: 40.62%] [G loss: 0.974284]
3774 [D loss: 0.464719, acc.: 81.25%] [G loss: 1.051173]
3775 [D loss: 0.691113, acc.: 59.38%] [G loss: 0.939521]
3776 [D loss: 0.738855, acc.: 46.88%] [G loss: 1.061326]
3777 [D loss: 0.754058, acc.: 43.75%] [G loss: 0.870659]
3778 [D loss: 0.648373, acc.: 62.50%] [G loss: 1.005190]
3779 [D loss: 0.694818, acc.: 59.38%] [G loss: 0.848825]
3780 [D loss: 0.782638, acc.: 40.62%] [G loss: 0.928426]
3781 [D loss: 0.686949, acc.: 59.38%] [G loss: 0.887636]
3782 [D loss: 0.608266, acc.: 75.00%] [G loss: 1.034044]
3783 [D loss: 0.682190, acc.: 6

3910 [D loss: 0.643366, acc.: 59.38%] [G loss: 0.893875]
3911 [D loss: 0.699509, acc.: 50.00%] [G loss: 1.009171]
3912 [D loss: 0.611888, acc.: 75.00%] [G loss: 0.762540]
3913 [D loss: 0.590362, acc.: 71.88%] [G loss: 0.929601]
3914 [D loss: 0.679919, acc.: 62.50%] [G loss: 0.832327]
3915 [D loss: 0.552719, acc.: 71.88%] [G loss: 0.937968]
3916 [D loss: 0.623709, acc.: 71.88%] [G loss: 0.989033]
3917 [D loss: 0.910677, acc.: 43.75%] [G loss: 0.942123]
3918 [D loss: 0.675390, acc.: 59.38%] [G loss: 1.162775]
3919 [D loss: 0.658141, acc.: 62.50%] [G loss: 1.021397]
3920 [D loss: 0.646949, acc.: 65.62%] [G loss: 0.948380]
3921 [D loss: 0.657669, acc.: 65.62%] [G loss: 0.935902]
3922 [D loss: 0.631275, acc.: 62.50%] [G loss: 1.102092]
3923 [D loss: 0.616094, acc.: 75.00%] [G loss: 1.093760]
3924 [D loss: 0.718828, acc.: 59.38%] [G loss: 1.060996]
3925 [D loss: 0.606219, acc.: 62.50%] [G loss: 1.102505]
3926 [D loss: 0.664489, acc.: 56.25%] [G loss: 0.928662]
3927 [D loss: 0.736824, acc.: 5